In [ ]:
#!pip install git+https://github.com/adapter-hub/adapter-transformers.git
!git clone https://github.com/huggingface/transformers
!pip install emoji
!pip install -U adapter-transformers

Cloning into 'transformers'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 54554 (delta 60), reused 81 (delta 18), pack-reused 54410
Receiving objects: 100% (54554/54554), 40.73 MiB | 30.47 MiB/s, done.
Resolving deltas: 100% (38115/38115), done.
     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=87f2b5a6d6ee7f416551206aee4f8babbcd3c0da9b279b7bd4b894352e34af97
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji
     |████████████████████████████████| 1.3MB 9.2MB/s 
     |████████████████████████████████| 2.9MB 31.2MB/s 
     |████████████████████████████████| 1.1MB 37.6MB/s 
     |████████████████████████████████| 890kB 66.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=8932

In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, EvalPrediction, BertTokenizerFast, AdapterType, AutoModelWithHeads
from transformers import Trainer,TrainingArguments, set_seed
import warnings

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", DeprecationWarning)

import tensorflow as tf
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, Dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

from transformers import (
    Trainer,
    TrainingArguments,
    )
import os
import re
import emoji
import random

import nltk
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, TweetTokenizer
from nltk.corpus import wordnet, stopwords

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'Tesla P100-PCIE-16GB'

In [ ]:
def preprocess(df):
    
    #removes URL
    pattern = r'https.?://[^\s]+[\s]?'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes usernames/mentions
    pattern = r'@[^\s]+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes emoji and smiley
    pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"
                         u"\U0001F300-\U0001F5FF"
                         u"\U0001F680-\U0001F6FF"
                         u"\U0001F1E0-\U0001F1FF"
                         u"\U00002500-\U00002BEF"
                         u"\U00002702-\U000027B0"
                         u"\U00002702-\U000027B0"
                         u"\U000024C2-\U0001F251"
                         u"\U0001f926-\U0001f937"
                         u"\U00010000-\U0010ffff"
                         u"\u2640-\u2642"
                         u"\u2600-\u2B55"
                         u"\u200d"
                         u"\u23cf"
                         u"\u23e9"
                         u"\u231a"
                         u"\ufe0f"
                         u"\u3030"
                         "]+", flags=re.UNICODE)
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes numbers
    pattern = r'\d+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes punctuation
    pattern = r"[^\w\s]"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)

    #removes stop words
    stop_words = stopwords.words("english")    
    remove_stop_words = lambda row: " ".join([token for token in row.split(" ")
                                              if token not in stop_words])
    df["tweet"] = df["tweet"].apply(remove_stop_words)
    
    #removes extra spaces
    pattern = r"[\s]+"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)
    
    return(df)

In [ ]:
def train_validate_test_split(df,seed= 2018, train_percent=.8, validate_percent=.125):
  train_tweet, test_tweet, train_label, test_label  = train_test_split(df.tweet.tolist(), df.label.tolist(), train_size=train_percent, stratify=df['label'])
  train_tweet, validate_tweet, train_label, validate_label = train_test_split(train_tweet, train_label, test_size=validate_percent, stratify=train_label)
  return train_tweet, validate_tweet, test_tweet, train_label, validate_label, test_label

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def model_initialize(model_name, num_labels = 2):
  model = AutoModelWithHeads.from_pretrained(model_name)
  model.add_adapter("sst-2", AdapterType.text_task)
  model.add_classification_head("sst-2", num_labels=num_labels)
  model.train_adapter(["sst-2"])
  model.set_active_adapters([["sst-2"]])

  # we freeze the model weights which in turn they wont be updated
  for param in model.base_model.parameters():
      param.requires_grad = False
  
  training_args = TrainingArguments(
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy='epoch',
    output_dir="./",
    overwrite_output_dir=True,
    learning_rate=1e-4,
    warmup_steps=500,
    weight_decay=0.01,
    do_train=True,
    do_eval=True,
    do_predict=True,
    num_train_epochs=5)
  
  set_seed(training_args.seed)
  return model, training_args

In [ ]:
def Train_model(model, training_args, train_dataset, val_dataset, test_dataset, test_label):
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset = train_dataset,
      eval_dataset = val_dataset)
  
  trainer.train()
  pred = trainer.predict(test_dataset)
  pred = np.argmax(pred[0],axis=1)
  print("The Weighted F1- Score is: ", f1_score(np.array(test_label), pred, average = 'weighted'))
  print("The Macro F1- Score is: ", f1_score(np.array(test_label), pred, average = 'macro'))
  print("Accuracy is: ", accuracy_score(np.array(test_label), pred))

In [ ]:
directory = "./"
languages = ['French', 'Arabic', 'English']
model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
for lang in languages:
	df = pd.read_csv(os.path.join(directory, lang+'.csv'))
	df = preprocess(df)
	train_tweet, validate_tweet, test_tweet, train_label, validate_label, test_label = train_validate_test_split(df,seed= 2018, train_percent=.8, validate_percent=.125)
 
	train_encodings = tokenizer(train_tweet, truncation=True, padding='max_length', max_length=128)
	val_encodings = tokenizer(validate_tweet, truncation=True, padding='max_length', max_length=128)
	test_encodings = tokenizer(test_tweet, truncation=True, padding='max_length', max_length=128)
	train_dataset = Dataset(train_encodings, train_label)
	val_dataset = Dataset(val_encodings, validate_label)
	test_dataset = Dataset(test_encodings, test_label)
 
	model, training_args = model_initialize(model_name, num_labels = 2)
	Train_model(model, training_args, train_dataset, val_dataset, test_dataset, test_label)
	# model.save_adapter(os.path.join("./", lang), "sst-2", with_head=True)
	print("="*200)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss
1,0.691411,0.646310
2,0.649923,0.631674
3,0.632139,0.629837
4,0.625068,0.624890
5,0.642587,0.635239


The Weighted F1- Score is:  0.5403407264545163
The Macro F1- Score is:  0.40196078431372545
Accuracy is:  0.6721311475409836


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss
1,0.494047,0.510176
2,0.489158,0.491935
3,0.466574,0.480244
4,0.472885,0.472351
5,0.468894,0.474924


The Weighted F1- Score is:  0.6951569873964752
The Macro F1- Score is:  0.44084983099951713
Accuracy is:  0.7884283246977547


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss
1,0.415525,0.431277
2,0.418477,0.412855
3,0.404766,0.405521
4,0.425957,0.404801
5,0.349258,0.405263


The Weighted F1- Score is:  0.7660603286094521
The Macro F1- Score is:  0.5566253423076104
Accuracy is:  0.8190742672863511


In [ ]:
# model.load_head("./Arabic")
# model.load_head("./French")
# model.load_head("./English")